In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_tuner import RandomSearch, HyperParameters

# Charger le jeu de données
df = pd.read_csv('../../../../Data/processed/data_pre_process.csv', index_col=0)

# Définition du chemin vers les images
path_to_images = '../../../../Data/raw/images/image_train/'

# Création des noms de fichier des images
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]

# Création des liens complets vers les images
df['lien'] = path_to_images + df['Nom image']

# Configuration de ImageDataGenerator pour le prétraitement et l'augmentation des données
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% des données pour la validation
)

# Générateur pour l'ensemble d'entraînement et de validation
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Aucun répertoire car les chemins sont complets dans `df['lien']`
    x_col='lien',  # La colonne contenant les chemins complets des images
    y_col='prdtypecode',  # La colonne contenant les étiquettes
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='lien',
    y_col='prdtypecode',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Fonction pour construire le modèle avec optimisation des hyperparamètres
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(64, 64, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(hp.Int('conv_layers', 1, 3)):  # Boucle ajoutant des couches Conv2D supplémentaires
        model.add(Conv2D(
            filters=hp.Int(f'filters_{i+2}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i+2}', values=[3, 5]),
            activation='relu'
        ))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(27, activation='softmax'))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Configuration et démarrage de la recherche d'hyperparamètres comme GridSearchCV 
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory="C:/Users/matth/OneDrive/Documents/GitHub/Rakuten_project/keras_tuner_dir",
    project_name='keras_tuner_demo'
)

tuner.search(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)]
)

# Récupération du meilleur modèle
best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(validation_generator)


c:\Users\matth\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


FileNotFoundError: [Errno 2] No such file or directory: '../../../../Data/processed/data_pre_process.csv'